# Using Tweepy to sniff out Twitter bots

### Load python tools

In [1]:
import pandas as pd
import numpy as np
import tweepy
import json

### Get authenticated

In [2]:
consumer_key="0EtWySJEum7PpQ7ubreQgLwH5"
consumer_secret="6DPNvqyx44VlJWL3AAdNj53hmkiNAh0uGrvjfVH0ixmyLLqQe7"
access_token="14790314-YvDi7tRrBMQNqPo2BEKqAx7n7lo8fBnA3InySZcg9"
access_token_secret="iKL0A2Hl5oBlbV3m0ffYGWsjK4EVnS9VxvjFRPCCfW5XZ"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
rt = api.retweets('1316821974115479552')

In [6]:
ID = 1316821974115479552

In [7]:
count = 100

### Search results for "VoteNoOnHolly"

In [8]:
results = api.search(q='VoteNoOnHolly', count=100)

### Get our list of retweets the @VoteNoOnHolly tweet

In [9]:
retweets_list = api.retweets(ID, count)

### Create a list of retweeters by screen_name

In [10]:
rters = []

for retweet in retweets_list: 
    rters.append(retweet.user.screen_name)

### Get user information for each retweeter

In [11]:
df_users = []

for r in rters:
    screen_name = r
    user = api.get_user(screen_name)
    df_users.append(user._json)

### Users into a dataframe

In [12]:
df = pd.DataFrame(df_users)